In [1]:
!pip install textblob
!pip install wordcloud
!python -m textblob.download_corpora
!pip install plotly
!pip install wordninja
!pip install pyspellchecker

  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached tqdm-4.61.0-py2.py3-none-any.whl (75 kB)
  Using cached regex-2021.4.4-cp39-cp39-manylinux2014_x86_64.whl (730 kB)
  Using cached wordcloud-1.8.1-cp39-cp39-manylinux1_x86_64.whl (363 kB)
  Using cached Pillow-8.2.0-cp39-cp39-manylinux1_x86_64.whl (3.0 MB)
  Using cached matplotlib-3.4.2-cp39-cp39-manylinux1_x86_64.whl (10.3 MB)
  Using cached numpy-1.20.3-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp39-cp39-manylinux1_x86_64.whl (1.2 MB)
[nltk_data] Downloading package brown to /home/daro/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/daro/nltk_data...
[nltk_data]   Package punkt 

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
import plotly.express as px

In [3]:
df = pd.read_csv("data/vaccination_all_tweets.csv", quotechar='"', delimiter=',').dropna()
df

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
6,1337851215875608579,Gunther Fehlinger,"Austria, Ukraine and Kosovo",End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False
9,1337842295857623042,Ch.Amjad Ali,Islamabad,#ProudPakistani #LovePakArmy #PMIK @insafiansp...,2012-11-12 04:18:12,671,2368,20469,False,2020-12-12 19:30:33,#CovidVaccine \n\nStates will start getting #C...,"['CovidVaccine', 'COVID19Vaccine', 'US', 'paku...",Twitter Web App,0,0,False
10,1337841934170255365,Tamer Yazar,Turkey-Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mar...","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78313,1392827719029649410,Inter Press Service,Also @ipsnoticias,IPS: News and Views from the Global South | Lo...,2008-11-26 14:46:55,19038,1582,6407,False,2021-05-13 13:03:00,#SputnikV Diplomacy: Russia makes sure to port...,['SputnikV'],TweetDeck,0,1,False
78314,1392826970979799041,yash,New Delhi,I exist here because democracy dies in darkness.,2017-09-22 04:42:00,146,850,321,False,2021-05-13 13:00:02,#SputnikV will be available in India from next...,"['SputnikV', 'BreakingNews']",Twitter for Android,0,0,False
78315,1392826919016488961,Global Tribune,New Delhi,India's Fearless National Mind,2020-02-22 06:35:12,60,8,15,False,2021-05-13 12:59:49,Sputnik V Could Be Available In Market From Ne...,"['SputnikV', 'vaccine', 'vaccination', 'NitiAa...",Twitter for Android,0,0,False
78317,1392826283608875012,Russian Embassy in Sri Lanka,Colombo,Embassy of the Russian Federation🇷🇺 in Sri Lan...,2011-08-01 00:55:43,4991,500,1617,True,2021-05-13 12:57:18,❗️The Food and Drug Authority of the Republic ...,"['Maldives', 'Russian', 'SputnikV', 'COVID19']",Twitter Web App,2,6,False
